In [1]:
!cd

f:\11yue\hospital_location


In [2]:
!python detect.py --weights best.pt --img 416 --conf 0.75 --source ./test2

requirements: F:\11yue\hospital_location\requirements.txt not found, check failed.


c:\Users\Phoenix\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
detect: weights=['best.pt'], source=./test2, data=data\coco128.yaml, imgsz=[416, 416], conf_thres=0.75, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  2022-10-13 Python-3.9.12 torch-1.13.0+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7020913 parameters, 0 gradients
image 1/13 F:\11yue\hospital_location\test2\AlAkbarFastFoodCorner.png: 320x416 1 hospital, 2 roads, 463.2ms
image 2/13 F:\11yue\hospital_location\test2\AliCafe.png: 320x416 

In [3]:
import glob
import re
import cv2
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
import csv

c:\Users\Phoenix\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
def preprocessing_image(filepath):
  img = cv2.imread(filepath) 
  img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR) 
  img = cv2.resize(img,(196,196))  
  img = img / 255
  return img

In [5]:
f = open('result.csv', 'w', encoding='UTF8')
writer = csv.writer(f)
header = ["img file path", "result"]
writer.writerow(header)

22

In [6]:
imagespath=glob.glob('./runs/detect/exp/*.png')
labelspath=glob.glob('./runs/detect/exp/labels/*.txt')

In [7]:
print(imagespath.__len__())
print(labelspath.__len__())

13
10


In [8]:
print(imagespath[0])

./runs/detect/exp\AlAkbarFastFoodCorner.png


In [9]:
imagespath[2][18:-3]

'Capstone_G18DrKNarendarbabu.'

In [10]:
# 0- busstop
# 1- highway 
# 2- hospital
# 3- road

In [15]:
lst_imagepath=[]
lst_result=[]

In [16]:
for c in range(0,200):     #imagespath.__len__()
    match = [s for s in labelspath if (imagespath[c][18:-3]+'txt') in s]
    # print(match)
    if match.__len__()==0:
        # row=[imagespath[0],'bad']
        # writer.writerow(row)
        lst_imagepath.append(imagespath[c])
        lst_result.append('bad')
    else:
        data=open(match[0],'r')
        data=data.read()
        data=data.split( )
        labels=[]
        for j in range(0,int((data.__len__())/5)):
            labels.append(data[j*5])
        # print(labels)
        busstop=labels.count('0')
        highway=labels.count('1')
        hospital=labels.count('2')
        road=labels.count('3')
        if highway==0:
            highway=0
        else:
            highway=1
        if road==0:
            road=0
        else:
            road=1    
        estimate=busstop+highway+road-hospital
        if estimate==1 or estimate==2:
            # filename=datas[5][25:-4]
            # imgpath='./runs/detect/exp/'+filename+'.png'
            imgs=preprocessing_image(imagespath[c])
            # ewe=cv2.imread(imgpath)
            # plt.imshow(ewe)
            # print(imgs.shape)       
            model = keras.models.load_model('./classification.h5')
            pred = model.predict(np.array([imgs]))
            pred = pred.reshape(-1)
            for i in pred:
                if i>=0.5:
                    bg=1
                else:
                    bg=-1
            estimate=estimate+bg
        # print(estimate)
        if estimate>=2:
            # row=[imagespath[i],'good']
            # writer.writerow(row)
            lst_imagepath.append(imagespath[c])
            lst_result.append('good')
        else:
            # row=[imagespath[i],'bad']
            # writer.writerow(row)
            lst_imagepath.append(imagespath[c])
            lst_result.append('bad')






1/1 [==============================] - 0s 111ms/step


In [17]:
print(lst_imagepath.__len__())
print(lst_result.__len__())

13
13


In [50]:
for i in range(0,lst_imagepath.__len__()):
    row=[lst_imagepath[i],lst_result[i]]
    writer.writerow(row)

In [51]:
f.close()